In [6]:
import pandas as pd
from pylab import *

FILE = "C:/Users/jtso/My Documents/BLCA1.txt"

maf = pd.read_table(FILE, skiprows = 3, usecols=["Hugo_Symbol", "Chromosome", "Start_position", "End_position", "Tumor_Sample_Barcode"])
maf.head()

,Hugo_Symbol,Chromosome,Start_position,End_position,Tumor_Sample_Barcode
0,KIAA0090,1,19549914,19549914,TCGA-BL-A0C8-01A-11D-A10S-08
1,COL9A2,1,40766912,40766912,TCGA-BL-A0C8-01A-11D-A10S-08
2,SLC6A9,1,44474107,44474107,TCGA-BL-A0C8-01A-11D-A10S-08
3,HSPB11,1,54395807,54395807,TCGA-BL-A0C8-01A-11D-A10S-08
4,ITGA10,1,145534100,145534100,TCGA-BL-A0C8-01A-11D-A10S-08


In [7]:
print len(maf)

39312


In [ ]:
FILE = "../hw6/hsa.gff3"
TABLE2 = "C:/Users/jtso/Downloads/targetScanS.gz"  # miRNA binding sites
TABLE3 = "C:/Users/jtso/Downloads/wgEncodeGencodePolyaV19 (1).gz"  # polyA sites
df = pd.read_table(FILE, comment="#", usecols=["Chromosome", "Start", "End", "Type", "miR Name"])

chr2df = {}
for chr in [str(i) for i in range(1,23)] + ["X", "Y"]:
    chr2df[chr] = df[df["Chromosome"] == "chr" + chr]
    
def pos2miR(chr, start, end):
    chunk = chr2df[str(chr)]  # shrink the search space to the given chr
    out = []
    for idx, row in chunk.iterrows():
        if row["Start"] <= start <= row["End"] or \
           row["Start"] <=  end  <= row["End"] or \
           (start < row["Start"] and row["End"] < end):
            # get the name (either hsa-mir-### or hsa-miR-###)
            miR = row["Type"] + ";" + str(row["Start"]) + ";" + str(row["End"]) + ";" + row["miR Name"].split("Name=")[1].split(";")[0]
            out.append(miR)
    return out

c = pd.DataFrame()
ms = pd.DataFrame()
me = pd.DataFrame()
rs = pd.DataFrame()
re = pd.DataFrame()
t = pd.DataFrame()
i = pd.DataFrame()
s = pd.DataFrame()
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    if row["Chromosome"] == "MT":
        continue
    hits = pos2miR(row["Chromosome"], (row["Start_position"]), (row["End_position"]))
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_position"], row["End_position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_position"]])
        me = me.append([row["End_position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1
        
hi = pd.read_table(TABLE2)

chrom2hi = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2hi[chr] = hi[hi["chrom"] == "chr" + chr]
    
def pos2miRbind(chr, start, end):
    area = chrom2hi[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["chromStart"] <= start <= row["chromEnd"] or \
           row["chromStart"] <= end <= row["chromEnd"] or \
           (start < row["chromStart"] and row["chromEnd"] < end):
            miR = "miR binding site" + ";" + str(row["chromStart"]) + ";" + str(row["chromEnd"]) + ";" + str(row["name"])
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    if row["Chromosome"] == "MT":
        continue
    hits = pos2miRbind(row["Chromosome"], row["Start_position"], row["End_position"])
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_position"], row["End_position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_position"]])
        me = me.append([row["End_position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1
        
fh = pd.read_table(TABLE3)
chrom2fh = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2fh[chr] = fh[fh["chrom"] == "chr" + chr]
    print "chr%s\thas %d  \t miR genes" % (chr, len(chrom2fh[chr]))

def pos2polyA(chr, start, end):
    area = chrom2fh[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["txStart"] <= start <= row["txEnd"] or \
           row["txStart"] <= end <= row["txEnd"] or \
           (start < row["txStart"] and row["txEnd"] < end):
            miR = str(row["name2"]) + ";" + str(row["txStart"]) + ";" + str(row["txEnd"]) + ";" + str(row["name"])
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    if row["Chromosome"] == "MT":
        continue
    hits = pos2polyA(row["Chromosome"], row["Start_position"], row["End_position"])
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_position"], row["End_position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_position"]])
        me = me.append([row["End_position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

LOC642587 1 209605571 209605571 TCGA-BT-A0YX-01A-11D-A10S-08 hits on miRNA_primary_transcript;209605478;209605587;hsa-mir-205
HGS 17 79660899 79660899 TCGA-DK-A1AF-01A-11D-A13W-08 hits on miRNA_primary_transcript;79660787;79660899;hsa-mir-6786
FOXF2 6 1390560 1390560 TCGA-DK-A3IQ-01A-31D-A20D-08 hits on miRNA_primary_transcript;1390549;1390646;hsa-mir-6720
FOXF2 6 1390560 1390560 TCGA-DK-A3IQ-01A-31D-A20D-08 hits on miRNA;1390559;1390580;hsa-miR-6720-3p
FOXF2 6 1390569 1390569 TCGA-DK-A3IQ-01A-31D-A20D-08 hits on miRNA_primary_transcript;1390549;1390646;hsa-mir-6720
FOXF2 6 1390569 1390569 TCGA-DK-A3IQ-01A-31D-A20D-08 hits on miRNA;1390559;1390580;hsa-miR-6720-3p
RTL1 14 101349412 101349412 TCGA-FD-A3SM-01A-11D-A22Z-08 hits on miRNA_primary_transcript;101349316;101349412;hsa-mir-127
ASH1L 1 155316194 155316194 TCGA-FD-A3SN-01A-12D-A22Z-08 hits on miRNA_primary_transcript;155316141;155316236;hsa-mir-555
MCPH1 8 6357451 6357451 TCGA-BL-A13J-01A-11D-A10S-08 hits on miR binding site;635745

In [ ]:
c.rename(columns={0:"Chrom"}, inplace =True)
ms.rename(columns={0:"mStart"}, inplace=True)
me.rename(columns={0:"mEnd"}, inplace=True)
rs.rename(columns={0:"rStart"}, inplace=True)
re.rename(columns={0:"rEnd"}, inplace=True)
t.rename(columns={0:"Type"}, inplace=True)
i.rename(columns={0:"ID"}, inplace=True)
s.rename(columns={0:"Tumor Sample Barcode"}, inplace=True)
final = pd.concat([c,ms,me,rs,re,t,i,s], axis=1)
final = final.reset_index(drop=True)
final

,Chrom,mStart,mEnd,rStart,rEnd,Type,ID,Tumor Sample Barcode
0,1,209605571,209605571,209605478,209605587,miRNA_primary_transcript,hsa-mir-205,TCGA-BT-A0YX-01A-11D-A10S-08
1,17,79660899,79660899,79660787,79660899,miRNA_primary_transcript,hsa-mir-6786,TCGA-DK-A1AF-01A-11D-A13W-08
2,6,1390560,1390560,1390549,1390646,miRNA_primary_transcript,hsa-mir-6720,TCGA-DK-A3IQ-01A-31D-A20D-08
3,6,1390560,1390560,1390559,1390580,miRNA,hsa-miR-6720-3p,TCGA-DK-A3IQ-01A-31D-A20D-08
4,6,1390569,1390569,1390549,1390646,miRNA_primary_transcript,hsa-mir-6720,TCGA-DK-A3IQ-01A-31D-A20D-08
5,6,1390569,1390569,1390559,1390580,miRNA,hsa-miR-6720-3p,TCGA-DK-A3IQ-01A-31D-A20D-08
6,14,101349412,101349412,101349316,101349412,miRNA_primary_transcript,hsa-mir-127,TCGA-FD-A3SM-01A-11D-A22Z-08
7,1,155316194,155316194,155316141,155316236,miRNA_primary_transcript,hsa-mir-555,TCGA-FD-A3SN-01A-12D-A22Z-08
8,8,6357451,6357451,6357451,6357458,miR binding site,ANGPT2:miR-224,TCGA-BL-A13J-01A-11D-A10S-08
9,11,62444099,62444099,62444099,62444106,miR binding site,LOC51035:miR-124/506,TCGA-DK-A3IU-01A-11D-A20D-08


In [1]:
final.to_excel("BLCAhits.xls")

NameError: name 'final' is not defined

In [ ]:
FILE = "C:/Users/jtso/My Documents/LUAD1.txt"

maf = pd.read_table(FILE, usecols=["Hugo_Symbol", "Chromosome", "Start_Position", "End_Position", "Tumor_Sample_Barcode"])

FILE = "../hw6/hsa.gff3"
TABLE2 = "C:/Users/jtso/Downloads/targetScanS.gz"  # miRNA binding sites
TABLE3 = "C:/Users/jtso/Downloads/wgEncodeGencodePolyaV19 (1).gz"  # polyA sites

df = pd.read_table(FILE, comment="#", usecols=["Chromosome", "Start", "End", "Type", "miR Name"])

chr2df = {}
for chr in [str(i) for i in range(1,23)] + ["X", "Y"]:
    chr2df[chr] = df[df["Chromosome"] == "chr" + chr]
    
def pos2miR(chr, start, end):
    chunk = chr2df[str(chr)]  # shrink the search space to the given chr
    out = []
    for idx, row in chunk.iterrows():
        if row["Start"] <= start <= row["End"] or \
           row["Start"] <=  end  <= row["End"] or \
           (start < row["Start"] and row["End"] < end):
            # get the name (either hsa-mir-### or hsa-miR-###)
            miR = str(row["Type"]) + ";" + str(row["Start"]) + ";" + str(row["End"]) + ";" + row["miR Name"].split("Name=")[1].split(";")[0]
            out.append(miR)
    return out

c = pd.DataFrame()
ms = pd.DataFrame()
me = pd.DataFrame()
rs = pd.DataFrame()
re = pd.DataFrame()
t = pd.DataFrame()
i = pd.DataFrame()
s = pd.DataFrame()
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

In [ ]:
hi = pd.read_table(TABLE2)

chrom2hi = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2hi[chr] = hi[hi["chrom"] == "chr" + chr]
    
def pos2miRbind(chr, start, end):
    area = chrom2hi[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["chromStart"] <= start <= row["chromEnd"] or \
           row["chromStart"] <= end <= row["chromEnd"] or \
           (start < row["chromStart"] and row["chromEnd"] < end):
            miR = "miR binding site" + ";" + str(row["chromStart"]) + ";" + str(row["chromEnd"]) + ";" + str(row["name"])
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

In [ ]:
fh = pd.read_table(TABLE3)
chrom2fh = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2fh[chr] = fh[fh["chrom"] == "chr" + chr]
    print "chr%s\thas %d  \t miR genes" % (chr, len(chrom2fh[chr]))

def pos2polyA(chr, start, end):
    area = chrom2fh[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["txStart"] <= start <= row["txEnd"] or \
           row["txStart"] <= end <= row["txEnd"] or \
           (start < row["txStart"] and row["txEnd"] < end):
            miR = str(row["name2"]) + ";" + str(row["txStart"]) + ";" + str(row["txEnd"]) + ";" + str(row["name"])
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

In [ ]:
c.rename(columns={0:"Chrom"}, inplace =True)
ms.rename(columns={0:"mStart"}, inplace=True)
me.rename(columns={0:"mEnd"}, inplace=True)
rs.rename(columns={0:"rStart"}, inplace=True)
re.rename(columns={0:"rEnd"}, inplace=True)
t.rename(columns={0:"Type"}, inplace=True)
i.rename(columns={0:"ID"}, inplace=True)
s.rename(columns={0:"Tumor Sample Barcode"}, inplace=True)
final = pd.concat([c,ms,me,rs,re,t,i,s], axis=1)
final = final.reset_index(drop=True)
final

In [ ]:
final.to_excel("LUADhits.xls")